# 023. Peroba pipeline
The `peroba` suite consists of 3 programs:
1. `peroba_database`: collects information from several sources and generates a set of `perobaDB` files
2. `peroba_backbone`: selects a set of "global" sequences from `perobaDB` to be analysed together with the local ones (NORW). It finds local sequences within the database, but the user should also include other local sequences. 
3. `peroba_report`: once the user finishes the analysis (i.e. has a phylogenetic tree using suggestions from `peroba_backbone`), this script will estimate ancestral states and generate a PDF report. Importantly it will estimate (impute) the lineages, uk_lineages and phylotypes for "all" local sequences that were not estimated by COGUK (except those failing QC, like excess of `N`s...)

### how to run the pipeline
There are two main usages for the pipeline. If we do not have new local sequences (i.e. that were not incorporated into the COGUK phylogenetics pipeline), then:
1. run `peroba_database` whenever the databases upstream (on COGUK) are updated. This usually happens by the beginning of the week (Tue or Wed)
2. run `peroba_report` using `perobaDB` global tree.
This will generate the report but will not impute lineages (since we are not giving extra information from our tree)

However if we have new sequences, we don't need to wait until they are analysed by COGUK:
1. Assume we have already ran `peroba_database` so that we have a `perobaDB` for the most recent data available (a few days old, from last Wednesday let's say). 
2. run `peroba_backbone` adding the local sequences. It will generate alignments and backbone trees to be used in the phylogenetic inference. It generates a NJ tree which can be used as a starting point but it's expected that the user then runs iqtree or raxml by hand. 
3. run `peroba_report` using the tree estimated in step 2 above.

The latter strategy has the advantage that `peroba` is more permissive than COGUK &mdash; that is, NORW sequences that were rejected by the phylogenetic pipeline at COGUK may still be included by `peroba_backbone` and thus have their lineages inferred. 

### caveats

* Please keep in mind that some sequences will never have a UK_lineage (or other classification) because they cannot be included in the phylogenetic analysis (e.g. too many `N`s confuse MAFFT and make the sample become a "rogue taxon"). 
* For more information about te COGUK phylogenetic pipeline, see further below on the notebook.
* Both the software and this tutorial are a work-in-progress. The command options and output may not be as shown here.
* this is a `bash` notebook (not `python`). Paths etc. won't be the same

In [1]:
# mkdir 0618 # please do this by hand outside the notebook; (`mkdir`+`cd` in loop is a bad combo)
cd 0618
pwd

/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/notebooks01/0618


***
## peroba_database

<div>
<img src="023_peroba_database.png" width="800"/>
</div>

In [2]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_database.py -h

usage: peroba_database [options] 

peroba_database is the script that generates from scratch the integrated data
from COGUK and GISAID

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           Print debugging statements
  -v, --verbose         Add verbosity
  -m csv [csv ...], --metadata csv [csv ...]
                        csv/tsv files with metadata information (from COGUK or
                        GISAID)
  -t treefile, --tree treefile
                        single treefile in newick format
  -s fas [fas ...], --fasta fas [fas ...]
                        fasta files with unaligned genomes from COGUK, GISAID
  -a aln [aln ...], --alignment aln [aln ...]
                        aligned sequences from previous iteration (speeds up
                        calculations)
  -i INPUT, --input INPUT
                        Directory where input files are. Default: working
                        directory
  -o OUTPUT, --output OUTPUT
           

In [10]:
# brief look at input files, downloaded from COGUK and GISAID
workfolder="${HOME}/Academic/Quadram/021.ncov"

ls -d ${workfolder}/01.COGUK/phylogenetics/*  # downloaded from CLIMB server
ls -d ${workfolder}/00.data/*

/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/alignments/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/civet/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/microreact/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/public/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/reports/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/trees/
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/0324.nextstrain.aligned.fasta.bz2
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/0324.nextstrain.tree_raw.nwk
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/0324.nextstrain.tree_raw.nwk.log
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/gadm36_GBR_1_sp.rds
/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/gadm36_GBR_2_sp.rds
/usr/users/QIB_fr005/deoli

***
`peroba_database` can receive several tables (metadata) from both GISAID and COGUK and it will merge them (based on colun called `sequence_name`). The tree must come from COGUK, and the sequences from GISAID and COGUK. 

The COGUK sequences must be unaligned and from the phylogenetics pipeline (the raw sequences have very non-standard names).

In [11]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_database.py -v \
 -m ${workfolder}/01.COGUK/phylogenetics/alignments/cog_2020-06-12_all_metadata.csv.gz \
    ${workfolder}/01.COGUK/phylogenetics/civet/cog_global_2020-06-12_metadata.csv.gz  \
    ${workfolder}/01.COGUK/phylogenetics/trees/cog_global_2020-06-12_metadata.csv.gz \
    ${workfolder}/00.data/gisaid.metadata_2020-06-03_06-11.tsv.gz \
 -t ${workfolder}/01.COGUK/phylogenetics/trees/cog_global_2020-06-12_tree.newick \
 -s ${workfolder}/00.data/gisaid.sequences_2020-06-03_06-11.fasta.xz \
    ${workfolder}/01.COGUK/phylogenetics/public/cog_2020-06-12.fasta.xz \
 -a ${workfolder}/200519.peroba/perobaDB.0608.sequences.aln.xz

perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/trees/cog_global_2020-06-12_tree.newick
perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/alignments/cog_2020-06-12_all_metadata.csv.gz
perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/civet/cog_global_2020-06-12_metadata.csv.gz
perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/01.COGUK/phylogenetics/trees/cog_global_2020-06-12_metadata.csv.gz
perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/gisaid.metadata_2020-06-03_06-11.tsv.gz
perobaDB 2020-06-18 14:33 [INFO] Found file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/00.data/gisaid.sequences_2020-06-03_06-11.fasta.xz
perobaDB 2020-06-18 14:33 [INFO] Found fi

ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
STEP 900 / 1000 (thread 0)                    
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

Combining ..
   done.                      

   done.                      

addsingle (nuc) Version 7.467
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
6 thread(s)


To keep the alignment length, 45 letters were DELETED.
To know the positions of deleted letters, rerun the same command with the --mapout option.

Strategy:
 FFT-NS-fragment (Not tested.)
 ?

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

peroba 2020

***

The output of `peroba_database` is what I call `perobaDB`, a set of files with a day timestamp:

In [12]:
ls

perobaDB.0618.html             perobaDB.0618.sequences.aln.xz
perobaDB.0618.metadata.csv.gz  perobaDB.0618.sequences.fasta.xz
perobaDB.0618.raw.csv.gz       perobaDB.0618.tree.nhx


***
## peroba_backbone

A common pitfall here is that the local sequences must be named `NORW-Exxxx` (where `Exxxx` is the sample hex code), which is not their original name when sequenced. 

<div>
<img src="023_peroba_backbone.png" width="800"/>
</div>

In [13]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_backbone.py -h

usage: peroba_backbone <perobaDB> [options]

peroba_backbone is the script that generates a global backbone data set
(COGUK+GISAID) given a local one (NORW). It depends on the prefix for a
perobaDB set of files (from `peroba_database`), like "perobaDB.0519". It's
recommended that you also local sequences, even without CSV metadata. You can
furthermore add a newick file with extra trees (the tree from previous run is
a good choice).

positional arguments:
  perobaDB

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           Print debugging statements
  -v, --verbose         Add verbosity
  -i INPUT, --input INPUT
                        Directory where perobaDB files are. Default: working
                        directory
  -c csv, --csv csv     csv table with metadata from NORW
  -s fasta.bz2, --sequences fasta.bz2
                        extra sequences from NORW
  -t , --trees          file with (user-defined) trees in newick format to
      

In [18]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_backbone.py -v perobaDB.0618 \
-s ${workfolder}/01.COGUK/NORW/NORW-20200610.fa.xz \
-t ${workfolder}/200519.peroba/peroba_backbone.0612_1119.user.trees.nhx

peroba_backbone 2020-06-18 15:02 [INFO] Reading metadata, sequences, and tree from peroba_database
peroba_backbone 2020-06-18 15:02 [INFO] Reading database metadata from '/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/perobaDB.0618.metadata.csv.gz'
peroba_backbone 2020-06-18 15:02 [INFO] Reading database tree from '/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/perobaDB.0618.tree.nhx'
peroba_backbone 2020-06-18 15:02 [INFO] Reading database sequences from '/usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/perobaDB.0618.sequences.aln.xz'
peroba 2020-06-18 15:02 [INFO] Read 47517 sequences from file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/perobaDB.0618.sequences.aln.xz
peroba_backbone 2020-06-18 15:02 [INFO] Finished loading the database; dataframe has dimensions (47517, 96) and it's assumed we have the same number of sequences; the tree may be smaller
peroba_backbone 2020-06-18 15:02 [INFO] R

peroba_backbone 2020-06-18 15:12 [INFO] Finished saving alignment
peroba_backbone 2020-06-18 15:12 [INFO] In total, 6662 sequences are part of the user-defined data set (user trees plus sequences found here)
peroba_backbone 2020-06-18 15:12 [INFO] Estimating NJ tree with all user-defined sequences (will be the first tree in file)
peroba_backbone 2020-06-18 15:12 [INFO] Finished saving user-defined trees to file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/peroba_backbone.0618_1502.user.trees.nhx
peroba_backbone 2020-06-18 15:12 [INFO] Saving data set with all local and selected global sequences
peroba_backbone 2020-06-18 15:12 [INFO] Saving sequences to file /usr/users/QIB_fr005/deolivl/Academic/Quadram/021.ncov/200404.UK/0618/peroba_backbone.0618_1502.norw-coguk.aln.xz
peroba_backbone 2020-06-18 15:12 [INFO] Finished saving alignment
peroba_backbone 2020-06-18 15:12 [INFO] Total of 5707 sequences will form the local+global data set
peroba_backbone 2020-06-18 1

In [19]:
ls

peroba_backbone.0618_1502.coguk.aln.xz
peroba_backbone.0618_1502.coguk.trees.nhx
peroba_backbone.0618_1502.norw.aln.xz
peroba_backbone.0618_1502.norw-coguk.aln.xz
peroba_backbone.0618_1502.norw-coguk.trees.nhx
peroba_backbone.0618_1502.user.aln.xz
peroba_backbone.0618_1502.user.trees.nhx
perobaDB.0618.html
perobaDB.0618.metadata.csv.gz
perobaDB.0618.raw.csv.gz
perobaDB.0618.sequences.aln.xz
perobaDB.0618.sequences.fasta.xz
perobaDB.0618.tree.nhx


***
## peroba_report

<div>
<img src="023_peroba_report.png" width="800"/>
</div>

In [17]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_report.py -h

usage: peroba_report [options]

peroba_report is the script that generates a PDF report given a tree and
metadata for new genomes

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           Print debugging statements
  -v, --verbose         Add verbosity
  -m csv.gz, --metadata csv.gz
                        metadata file formated by peroba
  -c csv, --csv csv     csv table with metadata from NORW
  -t treefile, --tree treefile
                        single treefile in newick format
  -p PREFIX, --prefix PREFIX
                        Date to be added to report
  -i INPUT, --input INPUT
                        Input directory with pandoc templates, if not using
                        default
  -o OUTPUT, --output OUTPUT
                        Output database directory. Default: working directory


In [20]:
python ~/Academic/Quadram/022.peroba/peroba/peroba_report.py -v \
-c ${workfolder}/01.COGUK/NORW/20200617.SARCOV2-Metadata.csv \
-t peroba_backbone.0618_1502.norw-coguk.trees.nhx \
-m perobaDB.0618.raw.csv.gz -o report.0618

peroba_report 2020-06-18 15:15 [INFO] Reading metadata (previously prepared by peroba)
peroba_report 2020-06-18 15:15 [INFO] Reading CSV file with metadata from NORW
peroba_report 2020-06-18 15:15 [INFO] Reading tree file and checking if there are duplicate names
peroba_report 2020-06-18 15:15 [INFO] 5707 leaves in treefile and metadata with shape (47821, 96)
peroba 2020-06-18 15:15 [INFO] Merging global metadata (COGUK+GISAID usually) with local one (NORW)
peroba_report 2020-06-18 15:15 [WARNING] 513	samples in local csv metadata were not found on tree (excluded from analysis due to low quality?)
peroba_report 2020-06-18 15:15 [INFO] 972	samples from local csv metadata were found on tree and will form the basis of the analysis
peroba_report 2020-06-18 15:15 [WARNING] Removing 6 unmapped leaves from tree dictionary
peroba_report 2020-06-18 15:15 [INFO] Will now prune these leaves from tree.
peroba_report 2020-06-18 15:16 [INFO] Comparing tree leaves to check for duplicates
peroba_repor

In [21]:
ls *

peroba_backbone.0618_1502.coguk.aln.xz
peroba_backbone.0618_1502.coguk.trees.nhx
peroba_backbone.0618_1502.norw.aln.xz
peroba_backbone.0618_1502.norw-coguk.aln.xz
peroba_backbone.0618_1502.norw-coguk.trees.nhx
peroba_backbone.0618_1502.user.aln.xz
peroba_backbone.0618_1502.user.trees.nhx
perobaDB.0618.html
perobaDB.0618.metadata.csv.gz
perobaDB.0618.raw.csv.gz
perobaDB.0618.sequences.aln.xz
perobaDB.0618.sequences.fasta.xz
perobaDB.0618.tree.nhx

report.0618:
csv_2020-06-18.csv.gz  metadata_2020-06-18.csv.gz  report_2020-06-18.html
figures/               pandoc.css                  report_2020-06-18.pdf
_htm_2020-06-18.md     _pdf_2020-06-18.md


## The COGUK pipeline
Every Fri at noon the COGUK collects all sequences (with metadata) submitted that week and start processing them &mdash; uploading them to GISAID and feeding it to the phylogenetic pipeline. Every week around Tue/Wed we then have a new files on the CLIMB server. Here is a description of the available files: 

```bash
Phylogenetics pipeline output published to `/cephfs/covid/bham/artifacts/published/latest/phylogenetics/`

Reports published to `reports/`

Unaligned (deduplicated, clean headers) COG sequences published to `alignments/cog_2020-06-19_all.fasta`

Aligned (deduplicated, clean headers) COG sequences published to `alignments/cog_2020-06-19_all_alignment.fasta`
Matching metadata published to `alignments/cog_2020-06-19_all_metadata.csv`

Filtered, aligned COG sequences published to `alignments/cog_2020-06-19_alignment.fasta`
Matching metadata with lineage information published to `alignments/cog_2020-06-19_metadata.csv`

Full, annotated tree published to `trees/cog_2020-06-19_tree.nexus`
Matching metadata published to `trees/cog_global_2020-06-19_metadata.csv`
UK lineage subtrees published in `trees/uk_lineages/`
UK lineage timetrees published in `trees/uk_lineages/`

Public tree published to `public/cog_global_2020-06-19_tree.newick`
Associated unaligned sequences published to `alignments/cog_2020-06-19_all.fasta`
Matching metadata with public fields only published to `public/cog_2020-06-19_metadata.csv`

Public tree for microreact published to `microreact/cog_global_2020-06-19_tree_public.newick`
Public metadata for microreact published to `microreact/cog_global_2020-06-19_metadata_public.csv`
Private metadata for microreact published to `microreact/cog_global_2020-06-19_metadata_private.csv`

Data for Civet published to `/cephfs/covid/bham/civet-cat/`

```
The particular files, and sometimes the table columns, change from week to week. As seen above, we do not use them all but I downloaded them anyway. using an LZMA algorithm (like `xz` or `tar -J`) which makes the files a gazillion times smaller. 